In [7]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
#wordnet_lemmatizer = WordNetLemmatizer()
pd.reset_option('display.float_format')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer 
wordnet_lemmatizer = WordNetLemmatizer()
pd.reset_option('display.float_format')
from sklearn.feature_extraction.text import CountVectorizer


In [8]:
cd /Users/chiragrakholiya/Downloads/Project2/

/Users/chiragrakholiya/Downloads/Project2


In [9]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

data = pd.read_csv('hn2018_2019.csv', parse_dates=['Created At'], date_parser=dateparse)
data.head()

,Unnamed: 0,Object ID,Title,Post Type,Author,Created At,URL,Points,Number of Comments
0,2294635,16043786,Top Hacker News Books in 2017,story,0x54MUR41,2018-01-01 00:06:16,https://hackernewsbooks.com/year/2017,1,0.0
1,2294636,16043817,Beijing enjoys best winter air quality in five...,story,gpetukhov,2018-01-01 00:15:13,http://www.scmp.com/news/china/policies-politi...,2,0.0
2,2294637,16043825,Ask HN: Any domain name registrars that don't ...,ask_hn,glockenspielen,2018-01-01 00:16:54,NaN,1,2.0
3,2294638,16043845,Controversial Therapy Has Led to Death Threats...,story,cpncrunch,2018-01-01 00:23:05,https://www.buzzfeed.com/tomchivers/inside-the...,2,0.0
4,2294639,16043851,Ruby 3x3 – Ruby 3 Will Be 3 Times Faster – Wha...,story,geraldbauer,2018-01-01 00:25:05,https://planetruby.github.io/advent2017/ruby3x3,2,0.0


<H3>Factorizing Our Target Class to Manage the Computation Power</H3>

In [5]:
#data['Postype_id'] = data["Post Type"].factorize()[0]
#category_id_df = data[["Post Type", 'Postype_id']].drop_duplicates().sort_values('Postype_id')
#category_to_id = dict(category_id_df.values)
#id_to_category = dict(category_id_df[['Postype_id', "Post Type"]].values)
#data.head()

In [6]:
category_to_id

NameError: name 'category_to_id' is not defined

<H3>Dividing into test and training class</H3>

In [10]:
type(data['Created At'][0])
split_date=pd.datetime(2019,1,1)
df_train = data.loc[data['Created At'] < split_date]
df_test = data.loc[data['Created At'] >= split_date]

In [11]:
Training_data=df_train["Title"]
Training_targets=df_train["Post Type"]
Testing_data=df_test["Title"]
Testing_targets=df_test["Post Type"]

In [12]:
def preprocess(sentence):
    sentence=sentence.encode("ascii", errors="ignore").decode()
    sentence = sentence.lower()
    #tokenizer = RegexpTokenizer(r'\w+')
    #tk = MWETokenizer([('ask', 'hn:'), ('show', 'hn:')])
    tokens = sentence.split()
    tokens=[i.strip('+-:?*_,~\'') for i in tokens]
    lemma_words=[wordnet_lemmatizer.lemmatize(w) for w in tokens]
    return lemma_words

In [13]:
count_vect = CountVectorizer(tokenizer=preprocess,stop_words=None)
X_train_counts = count_vect.fit_transform(Training_data)
X_train_counts

<276981x106423 sparse matrix of type '<class 'numpy.int64'>'
	with 2219598 stored elements in Compressed Sparse Row format>

In [14]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf

<276981x106423 sparse matrix of type '<class 'numpy.float64'>'
	with 2219598 stored elements in Compressed Sparse Row format>

In [15]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(276981, 106423)

In [16]:
clf = MultinomialNB(alpha=0.5, fit_prior=True, class_prior=None).fit(X_train_tfidf,Training_targets)

In [17]:
print(clf.predict(count_vect.transform(["Show HN Just Read – The best reader mode for Chrome"])))

['show_hn']


In [18]:
Testing_counts = count_vect.transform(Testing_data)
Testing_tfidf = tfidf_transformer.transform(Testing_counts)

predicted = clf.predict_proba(Testing_tfidf)
predicted_string = clf.predict(Testing_tfidf)

In [23]:
myfile = open('baseline-results.txt', 'w',buffering=4096)
for doc, pred_class, category , actual in zip(Testing_data.tolist(),predicted_string, predicted ,Testing_targets):
     myfile.write('%r  %s  %f  %f  %f  %f  %s %s' % (doc, pred_class ,category[3] ,category[0],category[2],category[1],actual,'right\n' if pred_class == actual else 'wrong\n'))
myfile.close()      

In [21]:
np.mean(predicted_string == Testing_targets)

0.9343147615056663